# Setup

In [1]:
import os
import openai
 
openai.api_key = os.environ["OPENAI_API_KEY"]

def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
      model="text-davinci-002",
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [2]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.FeverWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [3]:
import json
import sys

folder = './prompts/'
prompt_file = 'fever.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_prompt = prompt_dict['webthink_simple3']

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [4]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in idxs[:500]:
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

3687 Claim: Paramore is not from Tennessee.
Thought 1: I should search for Paramore, and see if it is from Tennessee.
Action 1: Search[Paramore]
Observation 1: Pages for logged out editors learn more. Paramore is an American rock band from Franklin, Tennessee, formed in 2004. The band currently consists of lead vocalist Hayley Williams, guitarist Taylor York and drummer Zac Farro. Williams and Farro are founding members of the group, while York, a high school friend of the original lineup, joined in 2007. The band is signed to Fueled by Ramen, a subsidiary of Atlantic Records, both owned by Warner Music Group.

Thought 2: The observation says that the band is "from Franklin, Tennessee", so the claim is false.
Action 2: Finish[REFUTES]
Observation 2: Episode finished, reward = 1


{'steps': 2, 'answer': 'REFUTES', 'gt_answer': 'REFUTES', 'question_idx': 3687, 'em': 1, 'reward': 1, 'f1': 1} 

1 1 1.0 4.081380128860474
-----------

6238 Claim: Eric Church is a sound mixer.
Thought 1: I ne

Thought 2: The observation says that he is a "writer, musicologist, clarinetist, saxophonist, and blues producer", but not a journalist.
Action 2: Finish[REFUTES]
Observation 2: Episode finished, reward = 1


{'steps': 2, 'answer': 'REFUTES', 'gt_answer': 'REFUTES', 'question_idx': 1551, 'em': 1, 'reward': 1, 'f1': 1} 

5 7 0.7142857142857143 4.018667697906494
-----------

2544 Claim: Tijuana is smaller than 3 other cities in Baja California.
Thought 1: I should search for Tijuana, and see if it is smaller than 3 other cities in Baja California.
Action 1: Search[Tijuana]
Observation 1: Pages for logged out editors learn more. Tijuana (US: /tiːˈhwɑːnə/ tee-HWAH-nə,[4][5][a] Spanish: [tiˈxwana] (listen)), known also by the initials T.J., is a city and municipal seat of Tijuana Municipality, Baja California, located on the Pacific Coast of Mexico. It is part of the San Diego-Tijuana metro area and the larger Southern California megapolitan area.. Tijuana traces its modern history to the a

Thought 3: The observation says that the town was founded in 1955, so the Mirny could not have traveled the globe before then.
Action 3: Finish[REFUTES]
Observation 3: Episode finished, reward = 0


{'steps': 3, 'answer': 'REFUTES', 'gt_answer': 'SUPPORTS', 'question_idx': 2825, 'em': 0, 'reward': 0, 'f1': 0} 

8 14 0.5714285714285714 4.261058500834873
-----------

3543 Claim: Andrew Kevin Walker is from North America.
Thought 1: I should search Andrew Kevin Walker and see if it says he is from North America.
Action 1: Search[Andrew Kevin Walker]
Observation 1: Pages for logged out editors learn more. Andrew Kevin Walker (born August 14, 1964) is an American screenwriter, producer and script doctor. He is known for having written Seven (1995), for which he earned a nomination for the BAFTA Award for Best Original Screenplay, as well as several other films, including 8mm (1999), Sleepy Hollow (1999) and many uncredited script rewrites.. Walker was born in Altoona, Pennsylvania, though s

Thought 4: The observation says that the film was released on 2 August 2019, so the film is a film.
Action 4: Finish[SUPPORTS]
Observation 4: Episode finished, reward = 1


{'steps': 4, 'answer': 'SUPPORTS', 'gt_answer': 'SUPPORTS', 'question_idx': 7026, 'em': 1, 'reward': 1, 'f1': 1} 

13 20 0.65 4.149456000328064
-----------

1720 Claim: Chris Eubank Jr. was an only child.
Thought 1: I need to search Chris Eubank Jr. and find if he was an only child.
Action 1: Search[Chris Eubank Jr.]
Observation 1: Pages for logged out editors learn more. Christopher Livingstone Eubank Jr (born 18 September 1989) is a British professional boxer. He has held multiple championships in two weight classes, including the WBA interim middleweight title twice between 2015 and 2021; the IBO super-middleweight title twice between 2017 and 2019; and the British middleweight title in 2016.. As of February 2022, he is ranked as the world's fourth best active middleweight by BoxRec,[3] seventh by the Transnation

Thought 1: I should search James VI and I, and see if it says he razed the Plantation of Ulster.
Action 1: Search[James VI and I]
Observation 1: Pages for logged out editors learn more. James VI and I (James Charles Stuart; 19 June 1566 – 27 March 1625) was King of Scotland as James VI from 24 July 1567 and King of England and Ireland as James I from the union of the Scottish and English crowns on 24 March 1603 until his death in 1625. The kingdoms of Scotland and England were individual sovereign states, with their own parliaments, judiciaries, and laws, though both were ruled by James in personal union.. James was the son of Mary, Queen of Scots, and a great-great-grandson of Henry VII, King of England and Lord of Ireland, and thus a potential successor to all three thrones. He succeeded to the Scottish throne at the age of thirteen months, after his mother was compelled to abdicate in his favour.

Thought 2: The observation does not say anything about James VI and I razing the Plant

...

Thought 2: The observation says that the film was "adapted by Alvin Sargent", so it supports the claim.
Action 2: Finish[SUPPORTS]
Observation 2: Episode finished, reward = 0


{'steps': 2, 'answer': 'SUPPORTS', 'gt_answer': 'REFUTES', 'question_idx': 4856, 'em': 0, 'reward': 0, 'f1': 0} 

255 471 0.5414012738853503 4.1155206367468375
-----------

5726 Claim: Emma Watson was born in the 15th of April 1990.
Thought 1: I should search Emma Watson and see when she was born.
Action 1: Search[Emma Watson]
Observation 1: Pages for logged out editors learn more. Emma Charlotte Duerre Watson (born 15 April 1990) is an English actress and activist. Known for her roles in both blockbusters and independent films, as well as for her women's rights work, she has received a selection of accolades, including a Young Artist Award and three MTV Movie Awards. Watson has been ranked among the world's highest-paid actresses by Forbes and Vanity Fair, and was named one of the 100 most influential people in

Thought 1: I need to search Tamerlan Tsarnaev and find if he was an only child.
Action 1: Search[Tamerlan Tsarnaev]
Observation 1: Pages for logged out editors learn more. Tamerlan Anzorovich Tsarnaev (/ˌtæmərˈlɑːn ˌtsɑːrˈnaɪɛf/; October 21, 1986 – April 19, 2013)[note 1] was an American-based Chechen terrorist and former boxer of Chechen and Avar descent [3][4] who, with his brother Dzhokhar Tsarnaev, planted pressure cooker bombs at the Boston Marathon on April 15, 2013.[1][5][6][7] The bombings killed three people and reportedly injured as many as 264 others.[8] He emigrated to the United States in 2004 at the age of 18.[9] At the time of the bombings, Tsarnaev was an aspiring boxer.[10]. Shortly after the Federal Bureau of Investigation declared them suspects in the bombings and released images of them, the Tsarnaev brothers killed an MIT policeman, carjacked an SUV, and engaged in a shootout with the police in the Boston suburb of Watertown. According to the federal indictment, du

Thought 2: Courtney Cox did not play Rachel Green.
Action 2: Finish[REFUTES]
Observation 2: Episode finished, reward = 1


{'steps': 2, 'answer': 'REFUTES', 'gt_answer': 'REFUTES', 'question_idx': 319, 'em': 1, 'reward': 1, 'f1': 1} 

263 483 0.5445134575569358 4.138894701596373
-----------

2543 Claim: John Dolmayan is Armenian-American.
Thought 1: I need to search John Dolmayan and find if he is Armenian-American.
Action 1: Search[John Dolmayan]
Observation 1: Pages for logged out editors learn more. John Dolmayan (Armenian: Ճոն Տոլմայեան; born July 15, 1972[1]) is a Lebanese-born Armenian-American musician, best known as the drummer of System of a Down. He is also the drummer for the band Indicator and former drummer for Scars on Broadway. Dolmayan ranked number 33 on Loudwire's list of Top 200 Hard Rock + Metal Drummers of All Time.[2]. John Dolmayan was born in Beirut, Lebanon, to Armenian parents.[3] During the Lebanese Civil War, his family moved to California.[4].

Thought 2: T

Thought 2: He has been nominated for an Academy Award, which is different from a British Academy of Film and Television Arts Award.
Action 2: Finish[REFUTES]
Observation 2: Episode finished, reward = 0


{'steps': 2, 'answer': 'REFUTES', 'gt_answer': 'SUPPORTS', 'question_idx': 6700, 'em': 0, 'reward': 0, 'f1': 0} 

265 490 0.5408163265306123 4.147479046607504
-----------

564 Claim: Rachel Green appeared in every episode of The Office.
Thought 1: I need to search Rachel Green and see if she appeared in every episode of The Office.
Action 1: Search[Rachel Green]
Observation 1: Pages for logged out editors learn more. Rachel Karen Green[1][2][3] is a fictional character, one of the six main characters who appeared in the American sitcom Friends. Portrayed by Jennifer Aniston, the character was created by David Crane and Marta Kauffman, and appeared in all of the show's 236 episodes during its decade-long run, from its premiere on September 22, 1994, to its finale on May 6, 2004. Introdu

Thought 2: The observation does not say if he is a Buddhist, so I cannot determine if the claim is true or not.
Action 2: Finish[NOT ENOUGH INFO]
Observation 2: Episode finished, reward = 1


{'steps': 2, 'answer': 'NOT ENOUGH INFO', 'gt_answer': 'NOT ENOUGH INFO', 'question_idx': 4744, 'em': 1, 'reward': 1, 'f1': 1} 

268 496 0.5403225806451613 4.1463181751389655
-----------

7203 Claim: Anne Rice spent her entire life in Brooklyn.
Thought 1: I need to search Anne Rice and find if she spent her entire life in Brooklyn.
Action 1: Search[Anne Rice]
Observation 1: Pages for logged out editors learn more. Anne Rice[1] (born Howard Allen Frances O'Brien; October 4, 1941 – December 11, 2021) was an American author of gothic fiction, erotic literature, and Christian literature.. She was best known for her series of novels The Vampire Chronicles. Books from The Vampire Chronicles were the subject of two film adaptations—Interview with the Vampire (1994) and Queen of the Damned (2002).. Born i